<a href="https://colab.research.google.com/github/rwfchan/ResumeScanner/blob/main/ResumeScannerBeta_1_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Input: the resumes folder path
ResumeScanner('File path to resume')

In [ ]:
# Connect personal google drive to Google Colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install --upgrade google-cloud-vision

In [ ]:
def sendCSVFile(csv_dataframe, dir):
    mydb = mysql.connector.connect(host="Enter Host Address", user="Enter Username", passwd="Enter Password", database="Enter Database Name")
    mycursor = mydb.cursor()

    csv_dataframe = csv_dataframe.drop(['Init_index'], axis=1)

    # get the jpg list
    jpgFiles = os.listdir(dir)
    i = 0
    
    # need to change the database name and table name
    sql_create_table = '''
                      CREATE TABLE csv_output.resume_scanner_beta
                      (Name TEXT(1000),Organization TEXT(1000),Position TEXT(1000),
                      Email TEXT(1000),Phone TEXT(1000),Skill TEXT(1000),
                      SchoolName TEXT(1000),SchoolDegree TEXT(1000),SchoolDate TEXT(1000),
                      Qualification TEXT(1000), Image LONGBLOB)
                      '''
    # need to change the database name and table name
    mycursor.execute("DROP TABLE IF EXISTS csv_output.resume_scanner_beta")
    mycursor.execute(sql_create_table)
    
    for row, column in csv_dataframe.iterrows():
      b = convertToBinaryData(os.path.join(dir, jpgFiles[i]))
      tup = (column.Name,
                 column.organization,
                 column.position,
                 column.email,
                 column.phone,
                 column.skill,
                 column.SchoolName,
                 column.SchoolDegree,
                 column.SchoolDate,
                 column.Qualifications,
                 b)
      # need to change the database name and table name
      mycursor.execute('''
                INSERT INTO csv_output.resume_scanner_beta 
                (Name, Organization, Position, Email, Phone, 
                Skill, SchoolName, SchoolDegree, 
                SchoolDate, Qualification, Image)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                ''',
                tup)
      i += 1
    
    mydb.commit()

In [ ]:
def detect_text_refine(path):
  """Detects text in the file."""
  from google.cloud import vision
  import io
  import os
  import pandas

  from enum import Enum
  
  class FeatureType(Enum):
      PAGE = 1
      BLOCK = 2
      PARA = 3
      WORD = 4
      SYMBOL = 5

  os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Upload Credential Here"
  
  client = vision.ImageAnnotatorClient()

  with io.open(path, 'rb') as image_file:
      content = image_file.read()

  image = vision.Image(content=content)

  response = client.document_text_detection(image=image)
  texts = response.full_text_annotation

  resume_blocks = []
  id = 1
  
  for page in texts.pages:
    for block in page.blocks:
      resume_block = {}
      resume_block['id'] = id
      resume_block['vertices'] = block.bounding_box.vertices
      resume_block['paragraphs'] = block.paragraphs
      id += 1
      if len(block.paragraphs) == 1 and len(block.paragraphs[0].words) == 1 and len(block.paragraphs[0].words[0].symbols) <= 2:
        continue
      else:
        resume_blocks.append(resume_block)
  
  width = get_resume_width(resume_blocks)
  
  #get the most left x-axis and the most top y-axis
  lx = 99999
  ty = 99999
  for block in resume_blocks:
    x = block['vertices'][0].x
    y = block['vertices'][0].y
    if lx > x:
      lx = x
    if ty > y:
      ty = y
  
  columns = []

  for block in resume_blocks:
    find = False
    for column in columns:
      if block['vertices'][0].x >= column[0]['vertices'][0].x  - 100 and block['vertices'][0].x  <= column[0]['vertices'][0].x + 100:
        column.append(block)
        find = True
        break
      
    if not find:
      column_blocks = []
      column_blocks.append(block)
      columns.append(column_blocks)
  
  #rearrange column blocks from left to right
  columns_ = []

  while columns:
    index = 0
    lt_x = columns[0][0]['vertices'][0].x
    for i in range(len(columns)):
      if lt_x > columns[i][0]['vertices'][0].x:
        index = i
        lt_x = columns[i][0]['vertices'][0].x
    columns_.append(columns[index])
    columns.pop(index)

  columns = columns_

  #pick the column which has the most blocks
  col_index = 0
  num_blocks = 0
  
  for i in range(len(columns)):
    if num_blocks < len(columns[i]):
      col_index = i
      num_blocks = len(columns[i])
  
  col_blocks = columns[col_index]

  #check if the reusme is one-column case or two-column case
  istwo = True

  num_realcolumns = 0
  for column in columns:
    if len(column) > 2:
      num_realcolumns += 1
  
  if num_realcolumns == 1:
    istwo = False

  if col_index == 0 and width < get_resume_width(col_blocks) + 30:
    istwo = False
    if len(columns) > 1:
      if get_density(col_blocks) > 0.35 and get_density(columns[col_index+1]) > 0.38 and get_blocks_len(col_blocks)/get_blocks_len(columns[col_index+1]) < 3:
        istwo = True
  
  if col_index != 0:
    if get_density(columns[col_index-1])<0.35 and len(columns[col_index-1]) >= 3:
      istwo = False

  print('is two column case?', istwo)

  if istwo:
    #check if the blocks in col_blocks are nested.
    #if not, then will insert blocks into gaps
    col_blocks_ = []

    if col_index != len(columns) - 1:
      if get_density(col_blocks) < 0.35:
        next_col_index = col_index+1
        next_col_blocks = columns[next_col_index]
        for i in range(len(col_blocks)-1):
          col_blocks_.append(col_blocks[i])
          del_idx = []
          for q in range(len(next_col_blocks)):
            if next_col_blocks[q]['vertices'][3].y <= col_blocks[i+1]['vertices'][1].y and next_col_blocks[q]['vertices'][0].y >= col_blocks[i]['vertices'][1].y-20:
              col_blocks_.append(next_col_blocks[q])
              del_idx.insert(0,q)
          for idx in del_idx:
            next_col_blocks.pop(idx)

        col_blocks_.append(col_blocks[-1])

        if next_col_blocks:
          for next_col_block in next_col_blocks:
            col_blocks_.append(next_col_block)
        
        col_blocks = col_blocks_

    #reorder blocks
    #1. delete col_blocks from resume blocks
    for col_block in col_blocks:
      id = col_block['id']
      for block in resume_blocks:
        if block['id'] == id:
          resume_blocks.remove(block)

    #2. combine col_blocks and resume_blocks and put in correct order
    if col_blocks[0]['vertices'][0].y < ty + 50 :
      if col_blocks[0]['vertices'][0].x < lx + 200:
        for block in resume_blocks:
          col_blocks.append(block)
        resume_blocks = col_blocks
      else:
        for block in col_blocks:
          resume_blocks.append(block)
    else:
      if col_blocks[0]['vertices'][0].x < lx + 100:
        for block in col_blocks:
          resume_blocks.append(block)
      else:
        last_blcoks = []
        for block in resume_blocks:
          if block['vertices'][0].x > col_blocks[0]['vertices'][0].x:
            last_blcoks.append(block)
            resume_blocks.remove(block)
        for block in col_blocks:
          resume_blocks.append(block)
        for block in last_blcoks:
          resume_blocks.append(block)
  
  #3. convert resume blcoks to output files
  i = 0   # Counting number of blocks
  x1, x2, y1, y2 = 9999, 0, 9999, 0   # the sentence's coordinates
  breaks = vision.TextAnnotation.DetectedBreak.BreakType
  bounds = [[[]]]
  sentence_location = []

  # Initialization
  bounds.append([])
  bounds[0].append([])

  for block in resume_blocks:
      s = 0   # Counting number of sentence
      for paragraph in block['paragraphs']:
          for word in paragraph.words:
              for symbol in word.symbols: # Beginning of word
                  pending_x1 = symbol.bounding_box.vertices[0].x
                  pending_y1 = symbol.bounding_box.vertices[0].y
                  pending_x2 = symbol.bounding_box.vertices[3].x
                  pending_y2 = symbol.bounding_box.vertices[3].y

                  x1 = pending_x1 if x1 > pending_x1 else x1
                  y1 = pending_y1 if y1 > pending_y1 else y1
                  x2 = pending_x2 if x2 < pending_x2 else x2
                  y2 = pending_y2 if y2 < pending_y2 else y2

                  if symbol.text == '•':
                    bounds[i][s] = "".join(bounds[i][s]) 
                    if (bounds[i][s] != '.'):
                        sentence_location.append((x1, y1, x2, y2))
                    bounds[i].append([])  # append new sentence
                    s+=1
                    x1, x2, y1, y2 = 9999, 0, 9999, 0   # reset the coordinates
                    continue
                  else: 
                    bounds[i][s]+=symbol.text
                  
                  if symbol.property.detected_break.type_ == breaks.SPACE:
                    bounds[i][s]+=' '  # End of a word
                  elif symbol.property.detected_break.type_ == breaks.EOL_SURE_SPACE or symbol.property.detected_break.type_ == breaks.LINE_BREAK:
                    bounds[i][s] = "".join(bounds[i][s]) 
                    if (bounds[i][s] != '.'):
                        sentence_location.append((x1, y1, x2, y2))
                    bounds[i].append([])  # append new sentence
                    s+=1
                    x1, x2, y1, y2 = 9999, 0, 9999, 0   # reset the coordinates
          bounds[i].append([])  # append new sentence
      i+=1
      bounds.append([])
      bounds[i].append([]) # For each sentence, not each paragraphs


  data = pandas.DataFrame(columns=['Sentence', 'Location'])

  tidy_text = ""
  s = 0
  for bound in bounds:
      for sentence in bound:
          if not sentence or sentence == '.':
            continue
          if (isinstance(sentence,list) == True):
            tidy_text = tidy_text + "".join(sentence) + '\n'
            sentence_location.insert(s, (-1,-1,-1,-1))    # If there are any out of handle line, simply insert invalid
            data = data.append({'Sentence' : "".join(sentence), 'Location' : (-1,-1,-1,-1)}, ignore_index=True)
          else:
            tidy_text = tidy_text + sentence + '\n'
            data = data.append({'Sentence' : sentence, 'Location' : sentence_location[s]}, ignore_index=True)
          s+=1
  sentence_location.append((0,0,0,0))   # Appending for end of resume indicator
  data = data.append({'Sentence' : "--------------------------------------------------\n", 'Location' : (0,0,0,0)}, ignore_index=True)
  
  return tidy_text, sentence_location, data

In [ ]:
pip install pdf2jpg

In [ ]:
pip install --upgrade spacy

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
pip install mysql-connector-python

In [ ]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir
from stanza.server import CoreNLPClient

In [ ]:
from nltk.tag.stanford import StanfordNERTagger
from nltk.tokenize import word_tokenize
import nltk

!wget 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip'
!unzip stanford-ner-2018-10-16.zip

nltk.download('punkt')

st = StanfordNERTagger('/content/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/content/stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')

In [ ]:
import spacy
from spacy.vocab import Vocab
import numpy
import pandas
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.models import load_model
import pickle
from spacy.matcher import Matcher
import pandas as pd
import re
import mysql.connector
from sqlalchemy import create_engine
import io
import os
import shutil
from pdf2jpg import pdf2jpg

In [ ]:
# Setup standford could
client = CoreNLPClient(annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner'], 
                    memory='4G', endpoint='http://localhost:9003', be_quiet=True)
import time; time.sleep(10)

2021-05-20 02:27:03 INFO: Writing properties to tmp file: corenlp_server-dab190ec56534389.props


In [ ]:
import mysql.connector
from mysql.connector import Error

def convertToBinaryData(filename):
    # Convert digital data to binary format
    import io
    from PIL import Image
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData

In [ ]:
def write_file(data, filename):
    # Convert binary data to proper format and write it on Hard Disk
    with open(filename, 'wb') as file:
        file.write(data)

In [ ]:
def convertPDF2JEG(directory):
  import os, shutil
  outputpathpre = "/content/drive/MyDrive/Resume Scanner Folder/ResumeFirstDir"
  outputpathpost = "/content/drive/MyDrive/Resume Scanner Folder/testingJPG"

  containPDF = False

  dir = outputpathpre
  for files in os.listdir(dir):
    path = os.path.join(dir, files)
    try:
        shutil.rmtree(path)
    except OSError:
        os.remove(path)
  
  dir = outputpathpost
  for files in os.listdir(dir):
    path = os.path.join(dir, files)
    try:
        shutil.rmtree(path)
    except OSError:
        os.remove(path)

  # Convert pdf to jpg and place them into separate directories each
  for filename in os.listdir(directory):
    if  filename.endswith(".pdf"):
      containPDF = True
      pdf2jpg.convert_pdf2jpg(os.path.join(directory, filename), outputpathpre, dpi=300, pages="ALL")

  # Place jpg from subdirectories into one singular directory
  for subdirectory in os.listdir(outputpathpre):
    for filename in os.listdir(os.path.join(outputpathpre, subdirectory)):
      shutil.move(os.path.join(outputpathpre, subdirectory, filename), outputpathpost)
  
  return containPDF


In [ ]:
def processFileFormat(directory):
  import os, shutil
  from pdf2jpg import pdf2jpg

  # Convert pdf to jpg and place them into the same directory
  for filename in os.listdir(directory):
    if  filename.endswith(".pdf"):
      pdf2jpg.convert_pdf2jpg(os.path.join(directory, filename), directory, dpi=300, pages="ALL")
      foldername = filename + '_dir'
      newpath = os.path.join(directory, foldername)

      for filename_ in os.listdir(newpath):
        newnamelist = filename_.split('.')
        newnamelist.remove('pdf')
        newname = '.'.join(newnamelist)
        os.rename(os.path.join(newpath,filename_),os.path.join(newpath,newname))
        shutil.move(os.path.join(newpath,newname),os.path.join(directory,newname))

      os.rmdir(newpath)
      os.remove(os.path.join(directory,filename))


In [ ]:
# Process the resume into relevant text informations
# Input: Directory of resumes
# Output: the name of the text files
def preprocess_resume(directory, debug=False):
    import os
    import re, string
    import copy

    print("Processing " + directory)

    outputtxt = "/content/BulkTxtOutput.txt"
    outputtxt_org = "/content/BulkTxtOutput_org.txt"

    # Set up file clean up
    newline = re.compile("[" + "\n" + "]")
    pattern = re.compile("[" + re.escape(string.punctuation) + "]")
    afterPattern = re.compile("•")
    text_orgPattern = re.compile("\"")

    text_file = open(outputtxt, "w")
    text_file_org = open(outputtxt_org, "w")
    data = pandas.DataFrame(columns=['Sentence', 'Location'])

    # Start by writing the column name into textfile
    text_file.write('Sentence' + '\n')
    text_file_org.write('Sentence' + '\n')

    # Counter for the first x resume we want to process
    i = 0
    x = 19

    #save process file name to csv
    filenameList = [[]]

    for filename in os.listdir(directory):
      if debug==True and i > x:
        break

      if not filename.endswith(".pdf"):
        print("\n" + "Currently processing: " + filename)
        filenameList += [filename]
        texts, sentence_location_perResume, df = detect_text_refine(os.path.join(directory, filename))
        df['Cleaned Sentence'] = df['Sentence']

        for row in range(len(df['Cleaned Sentence'])):
          if df['Cleaned Sentence'][row] != "--------------------------------------------------\n" :
            df['Cleaned Sentence'][row] = pattern.sub(' ', df['Cleaned Sentence'][row])
        for row in range(len(df['Sentence'])):
          df['Sentence'][row] = text_orgPattern.sub(' ', df['Sentence'][row])

        clean_sentence = '\n'.join(df['Cleaned Sentence'].tolist())
        text_file.write(clean_sentence)

        sentence = '\n'.join(df['Sentence'].tolist())
        text_file_org.write(sentence)

        data = data.append(df, ignore_index=True)

        print("Finished Resume number: " + str(i) + "--------------------------------------------------" + "\n")
        i+=1

    text_file.close()
    text_file_org.close()

    data = {'filename':filenameList}
    filenameDF = pd.DataFrame(data)
    filenameDF.to_csv('/content/fileorder.csv')

    return outputtxt, outputtxt_org

In [ ]:
def get_column_length(blocks):
  length = 0

  for block in blocks:
    l = block['vertices'][1].x - block['vertices'][0].x
    if l > length:
      length = l

  return length

In [ ]:
def get_density(blocks):
  cl = blocks[-1]['vertices'][3].y - blocks[0]['vertices'][0].y
  bl = 0
  for block in blocks:
    bl += block['vertices'][3].y - block['vertices'][0].y
  
  if not cl:
    return 1
    
  return bl/cl

In [ ]:
def get_resume_width(blocks):
  lx = 9999
  rx = 0
  for block in blocks:
    lx_ = block['vertices'][0].x
    rx_ = block['vertices'][1].x
    if lx_ < lx:
      lx = lx_
    if rx_ > rx:
      rx = rx_
  
  return rx - lx

In [ ]:
def get_blocks_len(blocks):
  bl = 0
  for block in blocks:
    bl += block['vertices'][3].y - block['vertices'][0].y
  
  return bl

In [ ]:
# Running Vision API for parsing resume image into text file
# Input: path of the resume image
# Output: the converted text string
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    import os

    from enum import Enum
    from PIL import Image
    im = Image.open(path)
    width, height = im.size
    # print(width)
    

    
    class FeatureType(Enum):
        PAGE = 1
        BLOCK = 2
        PARA = 3
        WORD = 4
        SYMBOL = 5

    # TODO: Modfiy this address
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Enter Credentials Here"
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)
    texts = response.full_text_annotation

    first_line_x = 0

    for page in texts.pages:
        for block in page.blocks:
            if block.bounding_box.vertices[0].x < width/2:
                first_line_x = block.bounding_box.vertices[0].x
                break

    i = 0   # Counting number of blocks
    x1, x2, y1, y2 = 9999, 0, 9999, 0   # the sentence's coordinates
    breaks = vision.TextAnnotation.DetectedBreak.BreakType
    bounds = [[[]]]
    sentence_location = []

    # Initialization
    bounds.append([])
    bounds[0].append([])

    num_sentence = 0

    for page in texts.pages:
        for block in page.blocks:
            s = 0   # Counting number of sentence
            if abs(block.bounding_box.vertices[0].x - first_line_x) < width/4:
                  for paragraph in block.paragraphs:
                      for word in paragraph.words:
                          for symbol in word.symbols: # Beginning of word
                              pending_x1 = symbol.bounding_box.vertices[0].x
                              pending_y1 = symbol.bounding_box.vertices[0].y
                              pending_x2 = symbol.bounding_box.vertices[3].x
                              pending_y2 = symbol.bounding_box.vertices[3].y

                              x1 = pending_x1 if x1 > pending_x1 else x1
                              y1 = pending_y1 if y1 > pending_y1 else y1
                              x2 = pending_x2 if x2 < pending_x2 else x2
                              y2 = pending_y2 if y2 < pending_y2 else y2

                              if symbol.text == '•':
                                bounds[i][s] = "".join(bounds[i][s]) 
                                if (bounds[i][s] != '.'):
                                    sentence_location.append((x1, y1, x2, y2))
                                bounds[i].append([])  # append new sentence
                                s+=1
                                x1, x2, y1, y2 = 9999, 0, 9999, 0   # reset the coordinates
                                continue
                              else: 
                                bounds[i][s]+=symbol.text
                              
                              if symbol.property.detected_break.type_ == breaks.SPACE:
                                bounds[i][s]+=' '  # End of a word
                              elif symbol.property.detected_break.type_ == breaks.EOL_SURE_SPACE or symbol.property.detected_break.type_ == breaks.LINE_BREAK:
                                bounds[i][s] = "".join(bounds[i][s]) 
                                if (bounds[i][s] != '.'):
                                    sentence_location.append((x1, y1, x2, y2))
                                bounds[i].append([])  # append new sentence
                                s+=1
                                x1, x2, y1, y2 = 9999, 0, 9999, 0   # reset the coordinates
                      bounds[i].append([])  # append new sentence
                  i+=1
                  bounds.append([])
                  bounds[i].append([]) # For each sentence, not each paragraphs
    
    for page in texts.pages:
        for block in page.blocks:
            s = 0   # Counting number of sentence
            if abs(block.bounding_box.vertices[0].x - first_line_x) >= width/4:
                  for paragraph in block.paragraphs:
                      for word in paragraph.words:
                          for symbol in word.symbols: # Beginning of word
                              pending_x1 = symbol.bounding_box.vertices[0].x
                              pending_y1 = symbol.bounding_box.vertices[0].y
                              pending_x2 = symbol.bounding_box.vertices[3].x
                              pending_y2 = symbol.bounding_box.vertices[3].y

                              x1 = pending_x1 if x1 > pending_x1 else x1
                              y1 = pending_y1 if y1 > pending_y1 else y1
                              x2 = pending_x2 if x2 < pending_x2 else x2
                              y2 = pending_y2 if y2 < pending_y2 else y2

                              if symbol.text == '•':
                                bounds[i][s] = "".join(bounds[i][s]) 
                                if (bounds[i][s] != '.'):
                                    sentence_location.append((x1, y1, x2, y2))
                                bounds[i].append([])  # append new sentence
                                s+=1
                                x1, x2, y1, y2 = 9999, 0, 9999, 0   # reset the coordinates
                                continue
                              else: 
                                bounds[i][s]+=symbol.text
                              
                              if symbol.property.detected_break.type_ == breaks.SPACE:
                                bounds[i][s]+=' '  # End of a word
                              elif symbol.property.detected_break.type_ == breaks.EOL_SURE_SPACE or symbol.property.detected_break.type_ == breaks.LINE_BREAK:
                                bounds[i][s] = "".join(bounds[i][s]) 
                                if (bounds[i][s] != '.'):
                                    sentence_location.append((x1, y1, x2, y2))
                                bounds[i].append([])  # append new sentence
                                s+=1
                                x1, x2, y1, y2 = 9999, 0, 9999, 0   # reset the coordinates
                      bounds[i].append([])  # append new sentence
                  i+=1
                  bounds.append([])
                  bounds[i].append([]) # For each sentence, not each paragraphs


    data = pandas.DataFrame(columns=['Sentence', 'Location'])

    tidy_text = ""
    s = 0
    for bound in bounds:
        for sentence in bound:
            if not sentence or sentence == '.':
              continue
            if (isinstance(sentence,list) == True):
              tidy_text = tidy_text + "".join(sentence) + '\n'
              sentence_location.insert(s, (-1,-1,-1,-1))    # If there are any out of handle line, simply insert invalid
              data = data.append({'Sentence' : "".join(sentence), 'Location' : (-1,-1,-1,-1)}, ignore_index=True)
            else:
              tidy_text = tidy_text + sentence + '\n'
              data = data.append({'Sentence' : sentence, 'Location' : sentence_location[s]}, ignore_index=True)
            s+=1
    sentence_location.append((0,0,0,0))   # Appending for end of resume indicator
    data = data.append({'Sentence' : "--------------------------------------------------\n", 'Location' : (0,0,0,0)}, ignore_index=True)
    
    return tidy_text, sentence_location, data

In [ ]:
# Reference from https://github.com/shabeelkandi/Handling-Out-of-Vocabulary-Words-in-Natural-Language-Processing-using-Language-Modelling/blob/master/RNN%20LSTM%20Model.ipynb

# generate a sequence using a language model
# Input: the model being used (rev or forward), tokenizer file, max_length of word, and seed of the text
# Output: list of predicted words
def generate_seq(model, tokenizer, max_length, seed_text):
    if seed_text == "":
        return ""
    else:
        in_text = seed_text
        n_words = 1
        n_preds = 5
        pred_words = ""
        # generate a fixed number of words
        for _ in range(n_words):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([in_text])[0]
            # pre-pad sequences to a fixed length
            encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
            # predict probabilities for each word
            proba = model.predict(encoded, verbose=0).flatten()
            #take the n_preds highest probability classes 
            yhat = numpy.argsort(-proba)[:n_preds] 
            # map predicted words index to word
            out_word = ''

            for _ in range(n_preds):
                for word, index in tokenizer.word_index.items():
                    if index == yhat[_]:
                        out_word = word
                        pred_words += ' ' + out_word
                        break

        return pred_words

In [ ]:
# Find and set embeddings for OOV words
# Input: the interested nlp string
# Output: None -- function set OOV to the global nlp vector
def set_embedding_for_oov(doc):
    #checking for oov words and adding embedding
    #Can improve accuracy performance by using the weight from embedding layer. 
    # However, still doesn't mean it will update the embedding layer
    for token in doc:
        if token.is_oov == True:
            before_text = doc[:token.i].text
            after_text = str(array(doc)[:token.i:-1]).replace('[','').replace(']','')

            pred_before = generate_seq(model, tokenizer, max_length-1, before_text).split()
            pred_after = generate_seq(rev_model, tokenizer, max_length-1, after_text).split()
            
            # Change embedding array size
            # spacy en_core_web_md uses 300 dimension. 
            # Even if the sentence max length becomes 40. We believe the max length of 
            embedding = numpy.zeros((300,))

            i=len(before_text)
            for word in pred_before:
                embedding += i*nlp.vocab.get_vector(word)
                i= i*.5
            i=len(after_text)
            for word in pred_after:
                embedding += i*nlp.vocab.get_vector(word)
                i= i*.25
            nlp.vocab.set_vector(token.text, embedding)

In [ ]:
#function to find most similar words
def most_similar(word):
    by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)
    return [w.orth_ for w in by_similarity[:10]]

In [ ]:
# Convert a line of words into array of vectors
# Input: the string of text
# Output: vectorized text string (size=300)
def line2vec(line, maxlength):
  import itertools

  separator = ' '
  # Make sure the embedding learn all the oov
  set_embedding_for_oov(nlp(separator.join(line)))
  # convert words into vectors
  vectors = [nlp.vocab.get_vector(word) for word in line]
  # join all vectors into one vector
  big_vector = itertools.chain(vectors)
  big_vector = pad_sequences(list(big_vector), maxlen=maxlength, dtype='float32', padding='post')

  return big_vector[0]

In [ ]:
# Error adjustment
# Input: data=pandas dataset of all the sentences; input=X; predictions=yhat
# Output: error corrected predictions
def HC_error_adjustment(data, input, predictions):
  header_list = ['academic', 'academics', 'experience', 'education', 'awards', 'award', 'courses', 'voluteer', 'volunteer', 'qualification', 'qualifications',
                 'professional', 'work', 'working', 'employment', 'internship', 'business', 'activities', 'project', 'projects', 'research', 'publication', 
                 'intern', 'languages', 'skills', 'skill', 'fluent', 'summary', 'objective', 'eductio', 'personal', 'profile',
                 'interest', 'personal', 'email', 'phone', 'milestones', 'linkedin', 'twitter', 'expertise', 'contact', 
                 'service', 'services', 'career', 'hobbies', 'coursework', 'traits', 'address', 'certificates', 'about', 'tel',
                 'university', 'organizations', 'organization', 'expert', 'certifications']
  header_black_list = ['leadership', 'speak', 'communities', 'enterprisse', 'administration', 'director', 'design', 'manager', 
                       'teacher', 'marketing', 'ethic', 'screenwriting', 'cooperative', 'excellence', 'kuo', 'writing', 'association',
                       'practices', 'university']

  for row in range(len(input)):
    if isinstance(data.at[row, 'Sentence'],float):
      continue
    
    string = data.at[row, 'Sentence'].replace(':', '')
    sentence = string.lower().split(' ')
    for item in header_list:
      if item in sentence:
        predictions[row] = 1
        break
    
    for item in header_black_list:
      if item in sentence:
        predictions[row] = 0
        break

    if sentence[0] != '' and string.split(' ')[0][0].islower():
      predictions[row] = 0
    
    if (len(sentence) > 5):
      predictions[row] = 0
    if (row == 0 
      or data.at[row-1, 'Sentence'] == '--------------------------------------------------'):
      predictions[row] = 1

  return predictions

In [ ]:
# Perpare for pulling all headers out and able to train on all papers (on BulkTxtOutput)
#   Program can pull all headers out, and put them into another csv
# Input: The resume in text file
# Output: file path of the csv file with headers marked
def get_headers(BulkTxtOutput, model=None):
  HC_model = 0
  if (model==None):
    HC_model = load_model('/content/drive/MyDrive/ECS 193A ResumeData/Model & Rev_Model/HC_model.h5')
  else:
    HC_model = load_model(model)

  csv_file = '/content/placeholding.csv'
  max_length = 300*5
  output_file = '/content/HC_output.csv'

  print('Loading BulkTxtFile...')
  # First, load text file to csv
  #read_file = pandas.read_csv (BulkTxtOutput)
  read_file = pandas.read_csv (BulkTxtOutput, sep="\n")
  read_file.to_csv (csv_file)

  print('Loading csv into dataframe...')
  # Now, load csv to pandas (because this is the way I know how to do this)
  data = pandas.read_csv(csv_file)
  data = data.drop(data.columns[0], axis=1)
  X = data.copy()

  print('Converting words into vectors...')
  for row in range(len(X['Sentence'])):
    if not isinstance(X['Sentence'][row],float):
      line = X['Sentence'][row].split()
    if len(line) > 5:
      line = line[:5]
    big_vector = line2vec(line, max_length)
    X['Sentence'][row] = numpy.asarray(big_vector).astype('float32')

  print('Done!')

  print('Perparing inputs for prediction model...')
  input = X['Sentence'].to_numpy().tolist()
  for row in range (len(input)):
    input[row] = input[row].tolist()
  
  print('Predicting...')
  predictions = (HC_model.predict_classes(input)).T[0]

  # Pick up missed traits
  predictions = HC_error_adjustment(data, input, predictions)
  
  print('Placing predictions into csv')
  output = pandas.DataFrame(columns=['Sentence'])
  # For every predictions[i] = 1, place the sentence onto the 
  for i in range (len(predictions)):
    if (predictions[i]):
      output.at[i, 'Sentence'] = data.at[i, 'Sentence']
  
  output.to_csv(output_file, index=True)

  return output_file



In [ ]:
# One_hot encoding
# Input: the classified output needed to be encode
# Output: One hot encoded output in numpy array
def one_hot_encoding(y):
  one_hot_y = np_utils.to_categorical(y)
  return one_hot_y.tolist()

In [ ]:
def isPhoneRow(text):
  import re, string
  pattern = re.compile("[" + re.escape(string.punctuation) + ' ' + "]")

  if isinstance(text, float) or '/' in text:
    return False

  sentence = pattern.sub('', text)

  if len(sentence) > 9 and sentence.isdecimal():
    return True

  return False

In [ ]:
def isWebsiteRow(text):
  if isinstance(text, float):
    return False

  email_list = ['com', 'gmail']
  
  sentence = text.lower().split(' ')
  if len(sentence) > 8:
    return False

  for item in email_list:
    if item in sentence:
      return True
        
  return False

In [ ]:
def isNameRow(text):
  if extract_name_new(text) != None:
    return True
  return False

In [ ]:
# error adjustment
# Input: data=pandas dataset of all the sentences; input=X; predictions=yhat
# Output: error corrected predictions
def LC_error_adjustment(data, input, predictions):
  label1_list = ['academic', 'academics', 'eductio', 'award', 'awards', 'education', 'coursework', 'courseworks', 
                 'qualification', 'qualifications', 'courses', 'course', 'university', 'certification']
  label2_list = ['professional', 'work', 'working', 'employment', 'internship', 'business', 'activities', 'project', 'intern', 'projects', 
                 'milestones', 'volunteer', 'service', 'services', 'experience', 'career', 'summary', 'objective', 'me', 'organization'
                 , 'organizations', 'research', 'publication']
  label3_list = ['languages', 'skills', 'skill', 'fluent', 'interest', 'expertise', 'hobbies', 'traits', 'proficiency', 'expert']
  label4_list = ['profile', 'personal', 'email', 'phone', 'linkedin', 'twitter', 'contact', 'address', 'tel']
  label5_list = ['reference']

  for row in range(len(input)):
    if (row == 0 
      or data.at[row-1, 'Sentence'] == '--------------------------------------------------'):
      predictions[row] = 4
      continue

    string = data.at[row, 'Sentence'].replace(':', '')
    sentence = string.lower().split(' ')
    for item in label2_list:
      if item in sentence:
        predictions[row] = 2
        break
    
    for item in label3_list:
      if item in sentence:
        predictions[row] = 3
        break
    
    for item in label1_list:
      if item in sentence:
        predictions[row] = 1
        break
    
    for item in label4_list:
      if item in sentence:
        predictions[row] = 4
        break
    
    #if isWebsiteRow(string) or isPhoneRow(string):
    #  print('@ LC_error_adjustment website, or phone detected at line:', row)
    #  predictions[row] = 4

    #if isNameRow(string) or isWebsiteRow(string) or isPhoneRow(string):
    #  print('@ LC_error_adjustment name, website, or phone detected at line:', row)
    #  predictions[row] = 4

    for item in label5_list:
      if item in sentence:
        predictions[row] = 5
        break
  
  return predictions

In [ ]:
# Note: labels with 5 could be ignore when labeling whole resumes
# Input: csv_file of the with Sentence only is ok
def get_label(csv_file, model=None, get_accuracy=False):
  LC_model = 0
  if (model==None):
    LC_model = load_model('/content/drive/MyDrive/ECS 193A ResumeData/Model & Rev_Model/LC_model.h5')
  else:
    LC_model = load_model(model)
  
  max_length = 300*5
  outputfile = '/content/LC_output.csv'

  print('Loading csv into dataframe...')
  # Load csv to pandas
  data = pandas.read_csv(csv_file)
  data.rename(columns={data.columns[0]: "Init_index"}, inplace=True)
  X = data.copy()

  print('Converting words into vectors...')
  for row in range(len(X['Sentence'])):
    line = X['Sentence'][row].split()
    if len(line) > 5:
      line = line[:5]
    big_vector = line2vec(line, max_length)
    X['Sentence'][row] = numpy.asarray(big_vector).astype('float32')

  print('Done!')

  print('Perparing inputs for prediction model...')
  input = X['Sentence'].to_numpy().tolist()
  for row in range (len(input)):
    input[row] = input[row].tolist()
  input = numpy.asarray(input)
  
  print('Predicting...')
  predictions = (LC_model.predict_classes(input))

  # Pick up missed traits
  predictions = LC_error_adjustment(data, input, predictions)

  # Prepare output as copy of data
  output = data.copy()

  # Append predictions onto output
  print('Placing predictions into csv')
  output['pred_Label'] = predictions
  
  # Check accuracy of this prediction
  if (get_accuracy == True):
    accuracy = 0
    correction_check = []
    for i in range (len(predictions)):
      if (predictions[i] == output.at[i, 'Label']):
        accuracy+=1
        correction_check.append(1)
      else:
        correction_check.append(0)
    
    output['Correction_check'] = correction_check
    print('Prediction accuracy: ', str(accuracy/len(predictions)*100))
    
  output.to_csv(outputfile, index=True)

  return outputfile

In [ ]:
# Input: string
# Output: boolean

def isQualification(words):
  #words = sentence.lower().split(' ')
  qualifications = ['level', 'levels', 'chartered', 'certified']
  if len(words) >= 2:
    for item in qualifications:
      if item in words:
        return True
  
  return False

In [ ]:
# Input: string
# Output: boolean

def isSchoolName(words):
  #words = sentence.lower().split(' ')
  schoolNames = ['university', 'college', 'school' , 'institute', 'polytechnic', 'cpa', 
                 'universiti', 'program', 'université', 'uc']
  specificNames = ['berkeley', 'san', 'irvine', 'los', 'davis', 'santa', 'riverside']
  namePhrase = [['chartered', 'financial', 'analyst']]
  schoolNamesBlackList = ['award', 'awarded', 'awards', 'president', 'leader', 'leaders', 'prize', 
                          'manager', 'middle', 'high', 'name', 'how']
  
  if len(words) == 1:
    for item in specificNames:
      if item in words:
        return True

  if len(words) >= 2:
    for item in schoolNamesBlackList:
      if item in words:
        return False

    for item in schoolNames:
      if item in words:
        return True

    # Check if each namePhrase is contained in sentence by
    #   ensure all strings in phrase also in array words
    #   if all strings are in array words, return True
    #   else, check the next phrase
    #   if exchange all phrases, return false
    for item in namePhrase:
      flag = True
      for subitem in item:
        if subitem not in words:
          flag = False
          break
      if flag:
        return flag
  
  return False
    


In [ ]:
# Input: string
# Output: boolean

def isSchoolDegree(words):
  schoolDegrees = ['bachelor', 'master', 'masters', 'minor', 'major', 'institute', 
                   'ba', 'ms', 'bs', 'bsc', 'b', 'msc', 'bba', 'hd', 'mba', 'ma', 'm',
                   'gce', 'level', 'levels', 
                   'diploma', 'business', 'degree', 'programme', 'chartered', 
                   'discipline', 'accounting', 'finance', 'course', 'coursework',
                   'science', 'computer', 'engineering', 'marketing', 'art', 'design', 'english']
  degreesphrase = [['foundations', 'program'], ['exchange', 'student']]
  schoolDegreesBlackList = ['award', 'awarded', 'awards', 'member', 'academic', 'academics', 
                            'prize', 'ranked', 'developed', 'prepared', 'technical', 'it', 'office',
                            'manager', 'relevant', 'gpa', 'i', 'name', 'hours', 'related', 'programmer']

  if len(words) >= 2:
    for item in schoolDegreesBlackList:
      if item in words:
        return False

    for item in schoolDegrees:
      if item in words:
        return True

    # Check if each degreesphrase is contained in sentence by
    #   ensure all strings in phrase also in array words
    #   if all strings are in array words, return True
    #   else, check the next phrase
    #   if exchange all phrases, return false
    for item in degreesphrase:
      flag = True
      for subitem in item:
        if subitem not in words:
          flag = False
          break
      if flag:
        return flag
  
  return False

In [ ]:
# Input: string
# Output: boolean

def isSchoolDate(words):
    schoolDateBlackList = ['gpa', 'scholarship', 'award', 'awarded', 'awards', 'out', 'dean', 'manager']

    if len(words) <= 4:
      for item in schoolDateBlackList:
        if item in words:
          return False

      for w in words:
        if w.isdigit():
          if len(w) == 1:
            return False
          else:
            return True
    return False

In [ ]:
# Input: Path of label 1 csv with lineNum and Sentence
# Output: None -- csv file with data picked out and grouped up

def extract_education(csv_file_path):
  import pandas as pd

  i = 0 # --> indexing for storage array

  startLine = []
  qualification, schoolName, schoolDegree, schoolDate = [[]], [[]], [[]], [[]]

  # Load csv Label1 into pandas
  data = pd.read_csv(csv_file_path)

  # Initalize beginning
  startLine.append(0)
  qualification.append([])
  schoolName.append([])
  schoolDegree.append([])
  schoolDate.append([])

  # For each rows in 'Sentence', place them into classified bins
  for row in range(len(data['Sentence'])):
    strings = data['Sentence'][row].split(' ')
    if data['Sentence'][row].split().count() >= 10 :
      continue
    # Parse sentence into subsentences
    subsentence = []
    subsentence.append([])
    j = 0
    for word in strings:
      if word == '':
        subsentence.append([])
        j+=1
      else:
        subsentence[j].append(word)
    for sentence in subsentence:
      # Skip empty cells
      if not sentence:
        continue
      if sentence[0] == "--------------------------------------------------":
        startLine.append(data[data.columns[0]][row])
        if row+1 != len(data['Sentence']):
          qualification.append([])
          schoolName.append([])
          schoolDegree.append([])
          schoolDate.append([])
        i+=1
      lower_sentence = [x.lower() for x in sentence]
      if isQualification(lower_sentence):
        qualification[i].append(' '.join(lower_sentence))
      elif isSchoolName(lower_sentence):
        schoolName[i].append(' '.join(lower_sentence))
      elif isSchoolDegree(lower_sentence):
        schoolDegree[i].append(' '.join(lower_sentence))
        print('School Degree Length:', data['Sentence'][row].split().count())
      elif isSchoolDate(lower_sentence):
        schoolDate[i].append(' '.join(lower_sentence))

  # Make data into directionary
  data = {'Init_index':startLine, 
          'SchoolName':schoolName,
          'SchoolDegree':schoolDegree,
          'SchoolDate':schoolDate,
          'Qualifications':qualification}
  
  # Convert data into dataFrame
  EducationList = pd.DataFrame(data)

  print(EducationList)

  # Put dataFrame into csv
  EducationList.to_csv("/content/EducationList.csv", index=True)

In [ ]:
# Load all the existing models


print('Loading word embedders')
# Models for word embedding
# If adding another path to load model, please add it to None, and comment out the existing load_model line
model = None
model = load_model('/content/drive/MyDrive/Resume Scanner Folder/WE_model.h5')
rev_model = None
rev_model = load_model('/content/drive/MyDrive/Resume Scanner Folder/WE_rev_model.h5')

# Saved tokenizer and maximum length of word
tokenizer_filepath = '/content/drive/MyDrive/Resume Scanner Folder/tokenizer.pkl'
with open(tokenizer_filepath, 'rb') as f:
    tokenizer = pickle.load(f)  
max_length_filepath = '/content/drive/MyDrive/Resume Scanner Folder/max_length.pkl'
with open(max_length_filepath, 'rb') as f:
    max_length = pickle.load(f)

#load spacy GloVe Model
#Can load en_core_web_lg for more unique vectors
nlp = spacy.load('en_core_web_md')

print('Complete')


Loading word embedders
Complete


In [ ]:
# Input: file path of clean text and unclean text
# Output: clean_unclean_text dataframe

def combine_text(outputtext,outputtext_org):
  clean = pandas.read_csv (outputtext, skip_blank_lines=False)
  unclean = pandas.read_csv (outputtext_org, sep="\n")

  clean_sentences = clean['Sentence']

  unclean.insert(1,"clean_Sentence",clean_sentences, True)
  
  return unclean

In [ ]:
# Input: file path of LC_output, clean_unclean_text dataframe
# Output: labeled_text df
# Labelize the combined text and save it as labeled_text.csv

def labelizingfulltext(lc_filepath, placeholding):
  #read LC_output.csv and placeholding.csv as datafram format
  lc_output = pandas.read_csv(lc_filepath)

  lcdf = lc_output.iloc[:,[1,3]]
  count_row = lcdf.shape[0]

  #replace Label 5 with the previous label in lc_output
  for i in range (1,count_row):
    if lcdf.iloc[i,1] == 5:
      lcdf.iloc[i,1] = lcdf.iloc[i-1,1]

  #get each line's label and save them into label list
  label = []
  for i in range(1,count_row):
    for j in range(0,lcdf.iloc[i,0]-lcdf.iloc[i-1,0]):   # lcdf.iloc[i-1,0] + j = currect row
      label = label + [lcdf.iloc[i-1,1]]

  label = label + [0]

  #Add label list to placeholding dataframe as a column
  placeholding.insert(2,"Label",label,True)
  placeholding = placeholding.rename({'Sentence': 'unclean_Sentence'}, axis=1)

  #Change all the lines that are website link or phone number to 4
  for i in range(1, placeholding.shape[0]):
    webstring = placeholding.at[i, 'clean_Sentence']
    phonestring = placeholding.at[i, 'unclean_Sentence']
    if isPhoneRow(phonestring) or isWebsiteRow(webstring):
      placeholding.at[i, "Label"] = 4

  #Save dataframe to Placeholding.csv
  placeholding.to_csv('labeled_text.csv')
  placeholding.insert(0,"index",numpy.arange(len(placeholding)))


  return placeholding

In [ ]:
def extract_title_corenlp(text):
  document = client.annotate(text)
  title_set = []
  for sent in document.sentence:
          for m in sent.mentions:
            if m.entityType == "TITLE" and text.count(" ") < 15:
              #title_set.append(m.entityMentionText)
              title_set.append(m.entityMentionText.lower())
  return title_set

In [ ]:
def extract_organization_corenlp(text):
  document = client.annotate(text)
  org_set = []
  for sent in document.sentence:
          for m in sent.mentions:
            if m.entityType == "ORGANIZATION" and text.count(" ") < 15:
              #org_set.append(m.entityMentionText)
              org_set.append(m.entityMentionText.lower())
  return org_set

In [ ]:
def extract_email(text):
    '''
    Helper function to extract email id from text
    :param text: plain text extracted from resume file
    '''
    email = re.findall(r"([^@|\s]+@[^@]+\.[^@|\s]+)", text)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [ ]:
def extract_mobile_number(text):

    mob_num_regex = r"\(?(\d{3})?\)?[\s\.-]{0,2}?(\d{3})[\s\.-]{0,2}(\d{4})"
    phone = re.findall(re.compile(mob_num_regex), text)
    
    if phone:
        number = ''.join(phone[0])
        return number

In [ ]:
def extract_name_revise(text):
  nlp = spacy.load("en_core_web_sm")
  doc2 = nlp(text)

  # Identify the persons
  persons = [ent.text for ent in doc2.ents if ent.label_ == 'PERSON']

  # Return persons
  if persons:
    persons = text

  return persons

In [ ]:
def extract_skills(nlp_text, noun_chunks, skills_file=None):
    '''
    Helper function to extract skills from spacy nlp text
    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param noun_chunks: noun chunks extracted from nlp text
    :return: list of skills extracted
    '''
    tokens = [token.text for token in nlp_text if not token.is_stop]
    if not skills_file:
        data = pd.read_csv(
            '/content/drive/MyDrive/Resume Scanner Folder/skills.csv'
        )
    else:
        data = pd.read_csv(skills_file)
    skills = list(data.columns.values)
    skillset = []
    # check for one-grams
    for token in tokens:
        if token.lower() in skills:
            #skillset.append(token)
            skillset.append(token.lower())

    # check for bi-grams and tri-grams
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    #return [i.capitalize() for i in set([i.lower() for i in skillset])]
    return [i.lower() for i in skillset]

In [ ]:
def extract_name_new(text):
  classified_text = st.tag(text.split())
  named_entities = get_continuous_chunks(classified_text)
  named_entities_str_tag = [(" ".join([token for token, tag in ne]), ne[0][1]) for ne in named_entities]
 
  for x in named_entities_str_tag:
    if x[1] == "PERSON":
      return x[0]
  return None

In [ ]:
def get_continuous_chunks(tagged_sent):
    continuous_chunk = []
    current_chunk = []

    for token, tag in tagged_sent:
        if tag != "O":
            current_chunk.append((token, tag))
        else:
            if current_chunk: # if the current chunk is not empty
                continuous_chunk.append(current_chunk)
                current_chunk = []
    # Flush the final current_chunk into the continuous_chunk, if any.
    if current_chunk:
        continuous_chunk.append(current_chunk)
    return continuous_chunk

In [ ]:
#Input: Path of laved clean text and unclean text file
#Output: None 

def extract_non_context(fulltext):
    import pandas as pd
    import timeit
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    i = 0

    startLine = []
    name, email, skill, phone, org, title = [[]], [[]], [[]], [[]], [[]], [[]]
    qualification, schoolName, schoolDegree, schoolDate = [[]], [[]], [[]], [[]]

    #get label4 and label1 sentences
    unclean_sentences = fulltext['unclean_Sentence']
    labels = fulltext['Label']
    clean_sentences = fulltext['clean_Sentence']
    info_sentences = []
    skill_sentences = []
    org_sentences = []
    education_sentence = []
    indx = []

    for row in range(len(unclean_sentences)):
     if labels[row] == 4 or labels[row] == 0:
       info_sentences.append(unclean_sentences[row])
       indx.append(fulltext.iloc[row][0])

    data = {'Init_index': indx,
            'unclean_Sentence': info_sentences}
    personalInfo = pd.DataFrame(data)

    indx = []
    for row in range(len(unclean_sentences)):
     if labels[row] == 3 or labels[row] == 0:
       skill_sentences.append(unclean_sentences[row])
       indx.append(fulltext.iloc[row][0])

    data = {'Init_index': indx,
            'unclean_Sentence': skill_sentences}
    skillInfo = pd.DataFrame(data)

    indx = []
    for row in range(len(unclean_sentences)):
     if labels[row] == 2 or labels[row] == 0:
       org_sentences.append(unclean_sentences[row])
       indx.append(fulltext.iloc[row][0])

    data = {'Init_index': indx,
            'unclean_Sentence': org_sentences}
    orgInfo = pd.DataFrame(data)

    indx = []
    for row in range(len(clean_sentences)):
     if labels[row] == 1 or labels[row] == 0:
       education_sentence.append(clean_sentences[row])
       indx.append(fulltext.iloc[row][0])
    
    data = {'Init_index': indx,
            'clean_Sentence': education_sentence}
    educationInfo = pd.DataFrame(data)

    #init beginning
    startLine.append(0)
    name.append([])
    org.append([])
    title.append([])
    email.append([])
    phone.append([])
    skill.append([])
    qualification.append([])
    schoolName.append([])
    schoolDegree.append([])
    schoolDate.append([])

    start = timeit.default_timer()

    #extract org and title
    print(len(orgInfo['unclean_Sentence']))
    for row in range(len(orgInfo['unclean_Sentence'])):
        string1 = orgInfo['unclean_Sentence'][row]
        if string1 == "--------------------------------------------------":
            startLine.append(orgInfo.iloc[row][1])
            if row+1 != len(orgInfo['unclean_Sentence']):
              org.append([])
              title.append([])
            i += 1

        orgtemp = extract_organization_corenlp(str(string1))
        titletemp = extract_title_corenlp(str(string1))

        if len(orgtemp) != 0:
            for oneorg in orgtemp:
                org[i].append(oneorg)
        if len(titletemp) != 0:
            for onetitle in titletemp:
                title[i].append(onetitle)
    
    end1 = timeit.default_timer()
    print('extract org and title Time:',end1 - start)

    #extract personal information
    i = 0
    startLine = []
    startLine.append(0)
    firstLine = personalInfo['unclean_Sentence'][0]
    perinfo_text = ''
    for row in range(len(personalInfo['unclean_Sentence'])):
        string1 = personalInfo['unclean_Sentence'][row]
        if string1 == "--------------------------------------------------":
            nametemp = extract_name_new(str(perinfo_text))
            perinfo_text = ''
            if nametemp and len(nametemp.split(' ')) <= 3:
              name[i].append(nametemp)
            if len(name[i]) < 1:
              name[i].append(firstLine)
            startLine.append(personalInfo.iloc[row][1])
            if row+1 != len(personalInfo['unclean_Sentence']):
              firstLine = personalInfo['unclean_Sentence'][row + 1]
              name.append([])
              email.append([])
              phone.append([])
            i += 1
        else:
            perinfo_text += ' ' + string1
        
        if len(email[i]) < 1:
          emailtemp = extract_email(str(string1))
          if emailtemp:
            email[i].append(emailtemp)
        
        if len(phone[i]) < 1:
          phonetemp = extract_mobile_number(str(string1))
          if phonetemp:
            phone[i].append(phonetemp)
    
    end2 = timeit.default_timer()
    print('extract personal information Time:',end2 - end1)

    #extract skill
    i = 0
    startLine = []
    startLine.append(0)

    nlp = spacy.load('en_core_web_sm')
    skill_text = ''

    for row in range(len(skillInfo['unclean_Sentence'])):
        string1 = skillInfo['unclean_Sentence'][row]
        
        if string1 == "--------------------------------------------------":
            tktxt = nlp(skill_text)
            skills_file = None
            noun_chunks = list(tktxt.noun_chunks)

            skilltemp = extract_skills(tktxt, noun_chunks, skills_file)

            if len(skilltemp) != 0:
                for oneskill in skilltemp:
                    skill[i].append(oneskill)
            
            skill_text = ''

            startLine.append(skillInfo.iloc[row][1])
            if row+1 != len(skillInfo['unclean_Sentence']):
              skill.append([])
            i += 1
        else:
            skill_text += ' ' + string1
    
    end3 = timeit.default_timer()
    print('extract skill Time:',end3 - end2)

    #extract education information
    i = 0
    startLine = []
    startLine.append(0)

    for row in range(len(educationInfo['clean_Sentence'])):
      strings = educationInfo['clean_Sentence'][row].split(' ')
      #  continue
      if educationInfo['clean_Sentence'][row].split().count(" ") > 9 :
        continue
      # Parse sentence into subsentences
      subsentence = []
      subsentence.append([])
      j = 0
      for word in strings:
        if word == '':
          subsentence.append([])
          j+=1
        else:
          subsentence[j].append(word)
      for sentence in subsentence:
        # Skip empty cells
        if not sentence:
          continue
        if sentence[0] == "--------------------------------------------------":
          startLine.append(educationInfo[educationInfo.columns[0]][row])
          if row+1 != len(educationInfo['clean_Sentence']):
            qualification.append([])
            schoolName.append([])
            schoolDegree.append([])
            schoolDate.append([])
          i+=1
        lower_sentence = [x.lower() for x in sentence]
        if len(lower_sentence) > 9:
          continue
        if isQualification(lower_sentence):
          qualification[i].append(' '.join(lower_sentence))
        elif isSchoolName(lower_sentence):
          schoolName[i].append(' '.join(lower_sentence))
        elif isSchoolDegree(lower_sentence):
          schoolDegree[i].append(' '.join(lower_sentence))
        elif isSchoolDate(lower_sentence):
          schoolDate[i].append(' '.join(lower_sentence))
    
    end4 = timeit.default_timer()
    print('extract education information Time:',end4 - end3)

    # Make data into directionary
    data = {'Init_index': startLine,
            'Name': name,
            'organization': org,
            'position': title,
            'email': email,
            'phone': phone,
            'skill': skill,
            'SchoolName':schoolName,
            'SchoolDegree':schoolDegree,
            'SchoolDate':schoolDate,
            'Qualifications':qualification}

    # Convert data into dataFrame
    noncontextualinfo = pd.DataFrame(data)
    noncontextualinfo = noncontextualinfo[:-1]

    # Put dataFrame into csv
    noncontextualinfo.to_csv("/content/Noncontutalinfo.csv", index=True)

    return noncontextualinfo

In [ ]:
# Note: might want to save nlp vectors
# Resume scanner (in Beta)
# Input: the Directory with resume of interest (The directory must only contain pdf files or JPG files)
# Output: the csv file with labels marked for each resumes
def ResumeScanner(directory):
  processFileFormat(directory)

  print('Converting resumes into text...')
  # Convert all resumes in directory to text fileR
  outputtext, outputtext_org = preprocess_resume(directory)
  print('Finished Converting')

  print('Picking out headers...')
  # Find the headers and output into csv
  #HC_model_path = None
  HC_model_path = '/content/drive/MyDrive/Resume Scanner Folder/HC_model.h5'
  HC_outputfile = get_headers(outputtext_org, HC_model_path)
  print('Finished Picking out headers')

  print('Picking out labels...')
  # Label each headers and output into csv
  #LC_model_path = None
  LC_model_path = '/content/drive/MyDrive/Resume Scanner Folder/LC_model.h5'
  LC_outputfile = get_label(HC_outputfile,LC_model_path)
  print('Finished Picking out labels')

  print('Combining text data...')
  combine_text_df = combine_text(outputtext, outputtext_org)
  print('Finished Combining text data')

  print('Lablizing text data...')
  labeled_text_df = labelizingfulltext(LC_outputfile,combine_text_df)
  print('Finished Lablizing text data...')

  print('Extracting information...')
  info_df = extract_non_context(labeled_text_df)
  info = pandas.read_csv('/content/Noncontutalinfo.csv')
  sendCSVFile(info, directory)
  print('Finished')